정수 리터럴과 덧셈만 있는 아주 간단한 산술식을 하스켈으로 다뤄보자.

In [1]:
:opt no-lint                          -- linter 끄기
{-# LANGUAGE ScopedTypeVariables #-}  -- 추가 언어 기능 설정

\noindent
문법구조
$\hspace{-2ex}\begin{array}{ll}
& n \in \texttt{Int} \\
& e \in \texttt{Expr} ~::=~ n ~\mid~ e + e
\end{array}$는 데이터 타입으로 선언하면 된다.

In [2]:
data Expr = I Int          -- n
          | Add Expr Expr  -- e + e
       deriving (Eq,Ord,Show) -- Eq 같은지, Ord 순서 비교, Show 출력 가능

작은걸음 의미구조\vspace*{-3ex}
$$
\frac{~ ~}{~ n_1 + n_2 \longmapsto n ~}
{\scriptstyle(\,n\;\equiv\;n_1\,+\,n_2\,)}
\quad
\frac{~ e_1 \longmapsto e_1' ~}{~ e_1 + e_2 \longmapsto e_1' + e_e ~}
\quad
\frac{~ e_1 \longmapsto e_1' ~}{~ e_1 + e_2 \longmapsto e_1' + e_e ~}
$$
는 모든 맥락에서 핵심 계산($n_1+n_2\longmapsto n$)을 진행하는 비결정적 의미구조이다.
우선 이 의미구조 그대로를 하스켈 프로그램으로 옮겨 다루어 보고,
그 다음에는 결정적 의미구조로 변형하는 방법에 대해 생각해 보자.

# 작은걸음 의미구조 검산기

In [3]:
Add (I n1) (I n2) ...> I n  =  n == n1 + n2
Add e1 e2 ...> Add e1' e2   =  e1 ...> e1'  -- 좌변에 변수 e2 중복
Add e1 e2 ...> Add e1  e2'  =  e2 ...> e2'  -- 좌변에 변수 e1 중복
_         ...> _            =  False  -- 다른 경우에는 진행 불가

: 

In [4]:
Add (I n1) (I n2) ...> I n  =  n == n1 + n2
Add e1 e2 ...> Add e1' e2'  =  e1 ...> e1' && e2 == e2'  ||
                               e2 ...> e2' && e1 == e1'
_         ...> _            =  False  -- 다른 경우에는 진행 불가

In [5]:
:type (...>)

(...>) :: Expr -> Expr -> Bool

In [6]:
Add (I 1) (I 2) ...> I 3  -- 검산 성공
I 1 `Add` I 2   ...> I 3
Add (I 1) (I 2) ...> I 4  -- 검산 실패

True

True

False

\noindent 참고로, 하스켈에서 전위(prefix) 표현인 생성자나 함수의 이름을 역따옴표($\,^{\backprime}$)로 감싸면 중위(infix) 연산자처럼 사용할 수 있다.

In [7]:
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 3)(I 3 `Add` I 4)
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 1 `Add` I 2)(I 7)
Add(I 1 `Add` I 2)(I 3 `Add` I 4) ...> Add(I 3)(I 7) -- 한꺼번에 2걸음

True

True

False

# 작은걸음 의미구조 계산기
비결정적인 작은걸음 관계($\longmapsto\;\subset\;\texttt{Expr}\times\texttt{Expr}$)를
함수($f_{\longmapsto}$)로 만드는 방법은 모든 가능한 다음 단계를 한꺼번에 모아놓은 집합을
결과값으로 하면 된다. 즉, 식($e\in\texttt{Expr}$) 하나에
식의 집합($\{e_1',e_2',\ldots,e_k'\}\in 2^\texttt{Expr}$) 하나를 대응시키는
함수 $f_{\longmapsto} : \texttt{Expr} \to 2^\texttt{Expr}$를
$f_{\longmapsto}(e) = \{\,e' \mid e\longmapsto e'\,\}$로 정의하면 된다.
이 정의는 단순명료하지만 작은걸음 의미구조의 계산기를 작성하는 데 적합하지 않다.
식 $e$의 문법구조를 따라 정의된 작은걸음 의미구조처럼 $f_{\longmapsto}$를
귀납적(inductive)으로 다음과 같이 정의하면 프로그램으로 옮기기에 좋다.
단, 마지막 등식에서 $e_1+e_2$는 위의 경우들과 겹치지 않는 형태($e_1+e_2\neq n_1+n_2$)라고 가정한다.
\vspace*{-2ex}
\begin{align*}
& f_{\longmapsto} ~ : ~ \texttt{Expr} \to 2^\texttt{Expr} \\
& f_{\longmapsto}(n)             &=~& \{\} \\
& f_{\longmapsto}(n_1 + n_2)     &=~& \{ n \} \qquad(n \equiv n_1 + n_2) \\
& f_{\longmapsto}(\,e_1 + e_2\,) &=~& \{ e_1' + e_2 \mid e_1\longmapsto e_1' \} ~ ~\, \cup ~ ~ \{ e_1 + e_2' \mid e_2\longmapsto e_2'\} \\
&                                &=~& \{ e_1' + e_2 \mid e_1'\in f_{\longmapsto}(e_1) \} \cup \{ e_1 + e_2' \mid e_2'\in f_{\longmapsto}(e_2)\}
\end{align*}
위 정의를 하스켈 리스트로 집합을 표현하는 `step` 함수로 옮기면 다음과 같다.

In [8]:
step :: Expr -> [Expr] -- 리스트로 집합을 표현
step (I n)             = []
step (I n1 `Add` I n2) = [ I n ]  where n = n1 + n2
step (Add e1 e2)       = [ Add e1' e2  | e1' <- step e1 ] ++
                         [ Add e1  e2' | e2' <- step e2 ]

In [9]:
step (I 3)                  -- step 함수 선언의 첫번째 등식에 패턴 매치
step (I 1 `Add` I 2)                       -- 두번째 등식에 패턴 매치 
step (Add (I 1 `Add` I 2) (I 3 `Add` I 4)) -- 세번째 등식에 패턴 매치

[]

[I 3]

[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]

## 하스켈 리스트

In [10]:
:type 'a'           -- 문자
:type ['a','b','c'] -- 문자 리스트 = 문자열(String)
:type False         -- 진리값
:type [False, True] -- 진리값 리스트 
:type (&&)          -- 함수
:type [(&&), (||)]  -- 함수 리스트

'a' :: Char

['a','b','c'] :: [Char]

False :: Bool

[False, True] :: [Bool]

(&&) :: Bool -> Bool -> Bool

[(&&), (||)] :: [Bool -> Bool -> Bool]

In [11]:
[True,False,'a','b'] -- 같은 타입끼리만 리스트 구성 가능하므로 오류

: 

### 리스트 이어붙이기 연산자
두 개의 리스트를 다음 연산자로 이어붙일 수 있다.

In [12]:
:type (++)

(++) :: forall a. [a] -> [a] -> [a]

In [13]:
[1,2,3] ++ [4,5,6] -- 정수 리스트 이어붙이기

[1,2,3,4,5,6]

$~$\vspace*{-2ex}\newline
리스트로 집합을 표현한다면 `++` 연산자는 합집합 연산에 해당한다.

참고로, 중위(infix) 연산자만 단독으로 괄호로 감싸면 전위(prefix) 표현인 함수 이름처럼 사용할 수 있다.

In [14]:
(++) [1,2,3] [4,5,6]

[1,2,3,4,5,6]

### 리스트 조건제시법
수학에서 집합 조건제시법(set comprehension)과 비슷한 형태의 문법인 리스트 조건제시법(list comprehension)으로 하스켈 리스트를 작성할 수 있다.

\indent
$\displaystyle\big\{(x,y) \mid x\in\{1,2\},\;y\in\{1,2,3\}\big\}$

In [15]:
[ (x,y) | x <- [1,2], y <-[1,2,3] ]

[(1,1),(1,2),(1,3),(2,1),(2,2),(2,3)]

\indent
$\displaystyle\big\{(x,y,z) \mid x,y,z\in\{1,\ldots,20\},\;x^2+y^2=z^2,\;x<y,\;\text{$x$와 $y$는 서로 소}\big\}$

In [16]:
[ (x,y,z) | x <- [1..20], y <- [1..20], z <- [1..20],
            x^2 + y^2 == z^2, x < y, gcd x y == 1 ]

[(3,4,5),(5,12,13),(8,15,17)]

In [17]:
[1..20]   -- 1부터 1씩 증가하는 20까지의 등차수열
[1,3..20] -- 원소 2개로 시작하면 등차수열 간격 설정

[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

[1,3,5,7,9,11,13,15,17,19]

## 여러 걸음(`step`) 진행하려면

In [18]:
e0 = (Add (I 1 `Add` I 2) (I 3 `Add` I 4))

step e0        -- (1+2)+(3+4)를 1걸음 진행

[Add (I 3) (Add (I 3) (I 4)),Add (Add (I 1) (I 2)) (I 7)]

In [19]:
step (step e0) -- (1+2)+(3+4)를 2걸음 진행???

: 

In [20]:
:type step

step :: Expr -> [Expr]

$~$\newline\noindent
함수 `step`은 산술식(`Expr`)에 적용해야 하는데 `step`의 결과는 산술식의 리스트(`[Expr]`)이므로
연쇄적으로 적용하려면 타입이 맞지 않는다. $g : A \to A$처럼 정의역과 공역이 같은 함수라야
$g(g(a))$와 같이 연쇄적으로 적용할 수 있다. 일반적으로 $f:A\to 2^A$처럼 하나의 원소를 같은 종류 원소들의 집합으로
대응시키는 함수의 정의역($A$)을 공역에 맞도록 확장한 버전의 함수를 $\hat{f}:2^A\to 2^A$는 다음과 같이 정의할 수 있다.
$$\hat{f}(S) = \bigcup_{a\in S} f(a)$$

In [21]:
-- (1+2)+(3+4) 계산 2걸음은, 1단계 진행 결과 각각에 step을 적용한 결과를 모아야
step (Add (I 3) (Add (I 3) (I 4)))
step (Add (Add (I 1) (I 2)) (I 7))

[Add (I 3) (I 7)]

[Add (I 3) (I 7)]